In [2]:
library(lmtest)
library(sandwich)
library(grf)
library(glmnet)
library(splines)
library(ggplot2)
library(reshape2)

Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Warning message:
"package 'grf' was built under R version 4.1.3"
Warning message:
"package 'glmnet' was built under R version 4.1.3"
Loading required package: Matrix

Loaded glmnet 4.1-3

Warning message:
"package 'reshape2' was built under R version 4.1.3"


In [13]:
set.seed(1) 
# \donttest{
# Train a causal forest.
n <- 50
p <- 10
X <- matrix(rnorm(n * p), n, p)
W <- rbinom(n, 1, 0.5)
Y <- pmax(X[, 1], 0) * W + X[, 2] + pmin(X[, 3], 0) + rnorm(n)
c.forest <- causal_forest(X, Y, W)

# Predict using the forest.
X.test <- matrix(0, 101, p)
X.test[, 1] <- seq(-2, 2, length.out = 101)
c.pred <- predict(c.forest, X.test)
# Estimate the conditional average treatment effect on the full sample (CATE).
average_treatment_effect(c.forest, target.sample = "all")
#>    estimate     std.err 
#> -0.09675011  0.40735444 

estimate    std.err 
0.07671321 0.40518288

In [16]:
c.pred

predictions
<dbl>
0.10296
0.10296
0.10296
0.10296
0.10296
0.10296
0.10296
0.10296
0.10296


In [15]:
# Estimate the conditional average treatment effect on the treated sample (CATT).
# We don't expect much difference between the CATE and the CATT in this example,
# since treatment assignment was randomized.
average_treatment_effect(c.forest, target.sample = "treated")
#>    estimate     std.err 
#> -0.06564791  0.40577566 

estimate   std.err 
0.1128299 0.3953098

In [ ]:
# Estimate the conditional average treatment effect on samples with positive X[,1].
average_treatment_effect(c.forest, target.sample = "all", subset = X[, 1] > 0)
#>  estimate   std.err 
#> 0.6126443 0.5750878 

# Example for causal forests with a continuous treatment.
n <- 2000
p <- 10
X <- matrix(rnorm(n * p), n, p)
W <- rbinom(n, 1, 1 / (1 + exp(-X[, 2]))) + rnorm(n)
Y <- pmax(X[, 1], 0) * W + X[, 2] + pmin(X[, 3], 0) + rnorm(n)
tau.forest <- causal_forest(X, Y, W)
tau.hat <- predict(tau.forest)
average_treatment_effect(tau.forest)
#>   estimate    std.err 
#> 0.41722579 0.02495619 
average_treatment_effect(tau.forest, subset = X[, 1] > 0)
#>   estimate    std.err 
#> 0.78518987 0.03564022 
# }